# LangChain 实战：服装销售聊天机器人

In [25]:
with open("real_costume_sales_data.txt", encoding="utf-8") as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [26]:
from langchain.text_splitter import CharacterTextSplitter

In [27]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 170,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [28]:
docs = text_splitter.create_documents([real_estate_sales])

In [29]:
docs[0]

Document(page_content='[顾客提问] 这件衣服质量怎么样？\n[销售回答] 这是我们店里的畅销款，采用的是高级面料，不仅手感舒适，而且耐穿耐洗，很多顾客都回购了。\n\n\\d+\\.\n[顾客提问] 这个价格有点贵，能便宜点吗？\n[销售回答] 我们的定价是根据产品的质量和设计来的，确实物有所值。而且，投资一件好衣服，长远来看更经济，因为它的耐用性会让您穿得更久。')

In [30]:
len(docs)

170

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [31]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

D:\000_application\010_appList\006_anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [32]:
query = "夏天买什么衣服"

In [33]:
answer_list = db.similarity_search(query)

In [34]:
for ans in answer_list:
    print(ans.page_content + "\n")

[顾客提问] 我想要一些适合夏天穿的衣服，有什么推荐？
[销售回答] 夏天穿着以清凉舒适为主，我们有多款轻薄透气的衣服，比如亚麻衬衫、棉质T恤等，都非常适合夏日穿着。

[顾客提问] 我想要一些适合夏季穿着的凉鞋。
[销售回答] 我们有多款适合夏季穿着的凉鞋，包括凉拖、凉鞋等，让您在炎热的夏天也能保持清爽舒适。

[顾客提问]这件衣服的面料是否适合夏季穿着？
[销售回答]这件衣服的面料选用了轻薄透气的材质，非常适合夏季穿着。它能够帮助您调节体温，保持干爽舒适。同时，我们品牌也注重面料的防晒性能，确保您在户外活动时也能得到良好的保护。

[顾客提问]这件衣服适合什么季节穿？
[销售回答]这款衣服的设计非常贴心，它采用了轻薄透气的面料，适合春秋季节穿着。无论是单穿还是作为内搭，都能轻松应对多变的天气。而且，它的颜色和设计都非常经典百搭，即使到了冬季，搭配一件外套也能让您时尚过冬。所以，无论您是哪个季节来选购，它都是个不错的选择哦。



In [35]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [36]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [37]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024CA77AC3A0>, search_kwargs={'k': 3})

In [38]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\000_application\010_appList\006_anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[顾客提问] 我想要一些适合夏天穿的衣服，有什么推荐？
[销售回答] 夏天穿着以清凉舒适为主，我们有多款轻薄透气的衣服，比如亚麻衬衫、棉质T恤等，都非常适合夏日穿着。

[顾客提问] 我想要一些适合夏季穿着的凉鞋。
[销售回答] 我们有多款适合夏季穿着的凉鞋，包括凉拖、凉鞋等，让您在炎热的夏天也能保持清爽舒适。

[顾客提问]这件衣服的面料是否适合夏季穿着？
[销售回答]这件衣服的面料选用了轻薄透气的材质，非常适合夏季穿着。它能够帮助您调节体温，保持干爽舒适。同时，我们品牌也注重面料的防晒性能，确保您在户外活动时也能得到良好的保护。



In [39]:
docs = topK_retriever.get_relevant_documents("衣服太贵了，能不能便宜点？")

In [40]:
for doc in docs:
    print(doc.page_content + "\n")

[顾客提问] 我对这件衣服的价格有些犹豫，能不能给个折扣？
[销售回答] 我们会定期举办促销活动，您可以关注我们的官方网站或者店铺，或者加入我们的会员享受专属折扣。

[顾客提问] 我对这件衣服的价格有些担忧，能否提供更多优惠？
[销售回答] 我们会定期推出促销活动，您可以关注我们的官方网站或者店铺，或者加入我们的会员享受专属优惠。

\d+\.
[顾客提问] 我想要一些适合旅行穿着的快干衣物。
[销售回答] 我们有专为旅行设计的快干衣物，它们轻便易折叠，适合旅行中的穿着和清洗。

[顾客提问] 这件衣服质量怎么样？
[销售回答] 这是我们店里的畅销款，采用的是高级面料，不仅手感舒适，而且耐穿耐洗，很多顾客都回购了。

\d+\.
[顾客提问] 这个价格有点贵，能便宜点吗？
[销售回答] 我们的定价是根据产品的质量和设计来的，确实物有所值。而且，投资一件好衣服，长远来看更经济，因为它的耐用性会让您穿得更久。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [41]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [42]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[顾客提问] 我想要一些适合夏天穿的衣服，有什么推荐？
[销售回答] 夏天穿着以清凉舒适为主，我们有多款轻薄透气的衣服，比如亚麻衬衫、棉质T恤等，都非常适合夏日穿着。

[顾客提问] 我想要一些适合夏季穿着的凉鞋。
[销售回答] 我们有多款适合夏季穿着的凉鞋，包括凉拖、凉鞋等，让您在炎热的夏天也能保持清爽舒适。

[顾客提问]这件衣服的面料是否适合夏季穿着？
[销售回答]这件衣服的面料选用了轻薄透气的材质，非常适合夏季穿着。它能够帮助您调节体温，保持干爽舒适。同时，我们品牌也注重面料的防晒性能，确保您在户外活动时也能得到良好的保护。



### 提取向量数据库中的`销售回答`

In [43]:
docs = retriever.get_relevant_documents(query)

In [44]:
docs[0].page_content

'[顾客提问] 我想要一些适合夏天穿的衣服，有什么推荐？\n[销售回答] 夏天穿着以清凉舒适为主，我们有多款轻薄透气的衣服，比如亚麻衬衫、棉质T恤等，都非常适合夏日穿着。'

In [45]:
docs[0].page_content.split("[销售回答] ")

['[顾客提问] 我想要一些适合夏天穿的衣服，有什么推荐？\n',
 '夏天穿着以清凉舒适为主，我们有多款轻薄透气的衣服，比如亚麻衬衫、棉质T恤等，都非常适合夏日穿着。']

In [46]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [47]:
ans

'夏天穿着以清凉舒适为主，我们有多款轻薄透气的衣服，比如亚麻衬衫、棉质T恤等，都非常适合夏日穿着。'

#### 尝试各种问题

In [48]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [49]:
query = "我想买厚一点的衣服"

print(sales(query))

[]


D:\000_application\010_appList\006_anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [50]:
print(sales(query, 0.8))

[]


D:\000_application\010_appList\006_anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [51]:
query = "我想买春天穿的衣服"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

D:\000_application\010_appList\006_anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []

score:0.75 ans: ['夏天穿着以清凉舒适为主，我们有多款轻薄透气的衣服，比如亚麻衬衫、棉质T恤等，都非常适合夏日穿着。', '我们有很多适合约会穿着的时尚款式，无论是优雅的连衣裙还是时尚的休闲装，都可以让您在约会中散发魅力。', '我们有多款适合冬天的保暖衣服，比如羊毛衫、羽绒服等，它们不仅保暖性强，而且款式多样，可以满足您的不同需求。', '[顾客提问]这件衣服适合什么季节穿？\n[销售回答]这款衣服的设计非常贴心，它采用了轻薄透气的面料，适合春秋季节穿着。无论是单穿还是作为内搭，都能轻松应对多变的天气。而且，它的颜色和设计都非常经典百搭，即使到了冬季，搭配一件外套也能让您时尚过冬。所以，无论您是哪个季节来选购，它都是个不错的选择哦。']

score:0.5 ans: ['夏天穿着以清凉舒适为主，我们有多款轻薄透气的衣服，比如亚麻衬衫、棉质T恤等，都非常适合夏日穿着。', '我们有很多适合约会穿着的时尚款式，无论是优雅的连衣裙还是时尚的休闲装，都可以让您在约会中散发魅力。', '我们有多款适合冬天的保暖衣服，比如羊毛衫、羽绒服等，它们不仅保暖性强，而且款式多样，可以满足您的不同需求。', '[顾客提问]这件衣服适合什么季节穿？\n[销售回答]这款衣服的设计非常贴心，它采用了轻薄透气的面料，适合春秋季节穿着。无论是单穿还是作为内搭，都能轻松应对多变的天气。而且，它的颜色和设计都非常经典百搭，即使到了冬季，搭配一件外套也能让您时尚过冬。所以，无论您是哪个季节来选购，它都是个不错的选择哦。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [52]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [53]:
qa_chain({"query": "有没有蚕丝做的衣服？"})

D:\000_application\010_appList\006_anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
D:\000_application\010_appList\006_anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '有没有蚕丝做的衣服？',
 'result': '蚕丝是一种古老且奢华的纺织材料，常用来制作各种服装。市面上确实有使用蚕丝制成的衣服，包括但不限于丝绸衬衫、裙子、睡衣、领带、围巾等。蚕丝衣物通常具有良好的透气性、柔软度和光泽，是高品质服装的选择之一。如果你在寻找蚕丝衣服，可以在服装店、奢侈品店或者在线零售商那里找到。'}

In [54]:
qa_chain({"query": "我穿衬衫好看吗？"})

D:\000_application\010_appList\006_anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我穿衬衫好看吗？',
 'result': '由于我无法看到你或你穿的衬衫，我无法判断你穿衬衫是否好看。通常来说，如果衬衫的款式、颜色和尺码适合你，且搭配得当，你穿衬衫应该会很好看。重要的是选择适合自己身型和风格的衣服，并穿着自信。'}

## 加载 FAISS 向量数据库已有结果

In [55]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [56]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

D:\000_application\010_appList\006_anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [57]:
qa_chain({"query": "我想买裤子，你们有么"})

D:\000_application\010_appList\006_anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买裤子，你们有么', 'result': '对不起，作为一个AI，我无法销售任何商品。我只能提供信息和帮助。'}

In [58]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [59]:
result = qa_chain({"query": "我想买别墅，你们有么"})

D:\000_application\010_appList\006_anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [60]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，我不能帮助您购买别墅。我是一个人工智能，主要用于提供信息和回答问题。',
 'source_documents': []}